In [1]:

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the "../input/" directory.
# For example, running this (by clicking run or pressing Shift+Enter) will list the files in the input directory

import os
# print(os.listdir("../input"))

# Any results you write to the current directory are saved as output.
import itertools
import os

%matplotlib inline
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import tensorflow as tf

from sklearn.preprocessing import LabelBinarizer, LabelEncoder
from sklearn.metrics import confusion_matrix

from tensorflow import keras
layers = keras.layers
models = keras.models


# This code was tested with TensorFlow v1.8
print("You have TensorFlow version", tf.__version__)

You have TensorFlow version 2.1.0


In [2]:
url = r"C:\Users\gonza\OneDrive\Documentos\datascience-thebridge-master\ejercicios gonzalo\DATA\bbc-text.csv"
data = pd.read_csv(url)
data.head(4)

,category,text
0,tech,tv future in the hands of viewers with home th...
1,business,worldcom boss left books alone former worldc...
2,sport,tigers wary of farrell gamble leicester say ...
3,sport,yeading face newcastle in fa cup premiership s...


In [3]:
data["category"].value_counts(normalize=True)

sport            0.229663
business         0.229213
politics         0.187416
tech             0.180225
entertainment    0.173483
Name: category, dtype: float64

In [4]:
train_size = int(len(data) * .8)
print(f"Train size: {train_size}")
print(f"Test size: {len(data)-train_size}")

Train size: 1780
Test size: 445


In [5]:
def train_test_split(data, train_size):
    train = data[:train_size]
    test = data[train_size:]
    return train, test

### Data preparation
There's some work to be done in order for our data to be ready for training.

#First we'll split the data into training and test sets.
#Then we'll tokenize the words (text), and then convert them to a numbered index.
#Next we'll do the same for the labels (categories), by using the LabelEncoder utility.
#Finally, we'll convert the labels to a one-hot representation.

In [6]:
train_cat, test_cat = train_test_split(data["category"], train_size)
train_text, test_text = train_test_split(data["text"],train_size)

In [7]:
max_words = 1000
tokenize = keras.preprocessing.text.Tokenizer(num_words=max_words, char_level=False)

In [8]:
tokenize.fit_on_texts(train_text) # fit tokenizer to our trian data
x_train = tokenize.texts_to_matrix(train_text)
x_test = tokenize.texts_to_matrix(test_text)

In [10]:
# use sklearn to convert label strings to numbered index (ojo solo usamos transform en test no fit)
encoder = LabelEncoder()
encoder.fit(train_cat)
y_train = encoder.transform(train_cat)
y_test = encoder.transform(test_cat)

In [16]:
# converts the labels to a one hot represantion
num_classes = np.max(y_train) +1 
y_train = keras.utils.to_categorical(y_train, num_classes)
y_test = keras.utils.to_categorical(y_test, num_classes)

In [17]:
# dimension of our training and test data
print ("x_train shape:", x_train.shape)
print ("x_test shape:", x_test.shape)
print ("y_train shape:", y_train.shape)
print ("y_test shape:", y_test.shape)

x_train shape: (1780, 1000)
x_test shape: (445, 1000)
y_train shape: (1780, 5)
y_test shape: (445, 5)


In [18]:
# This model trains very quickly and 2 epochs are already more than enough
# Training for more epochs will likely lead to overfitting on this dataset
# You can try tweaking these hyperparamaters when using this model with your own data
batch_size = 32
epochs = 2
drop_ratio = 0.5

In [20]:
# build the model
model = models.Sequential()
model.add(layers.Dense(512,input_shape=(max_words,)))
model.add(layers.Activation("relu"))

model.add(layers.Dense(num_classes))
model.add(layers.Activation("softmax"))

model.compile(loss="categorical_crossentropy",
                optimizer="adam",
                metrics=["accuracy"])


In [21]:
# model.fit trains the model
# The validation_split param tells Keras what % of our training data should be used in the validation set
history = model.fit(x_train, y_train,
                    batch_size=batch_size,
                    epochs=epochs,
                    verbose=1,
                    validation_split=0.1)

Train on 1602 samples, validate on 178 samples
Epoch 1/2
1602/1602 [==============================] - 2s 1ms/sample - loss: 0.4182 - accuracy: 0.8876 - val_loss: 0.1291 - val_accuracy: 0.9551
Epoch 2/2
1602/1602 [==============================] - 1s 357us/sample - loss: 0.0560 - accuracy: 0.9863 - val_loss: 0.1061 - val_accuracy: 0.9607


In [22]:
# Evaluate the accuracy of our trained model
score = model.evaluate(x_test, y_test, batch_size=batch_size,verbose=1)
print("test loss:", score[0])
print("test accuracy:", score[1])

445/445 [==============================] - 0s 211us/sample - loss: 0.1415 - accuracy: 0.9551
test loss: 0.14149320239431404
test accuracy: 0.9550562


## Make some predictions
Take some samples from the test dataset and inspect some individual predictions, to ensure that things are sensible.

In [26]:
# Here's how to generate a prediction on individual examples
text_labels = encoder.classes_

for i in range(10):
    prediction = model.predict(np.array([x_test[i]]))
    predicted_label = text_labels[np.argmax(prediction)]
    print (test_text.iloc[i][:50], "...")
    print ("Actual label:",  test_cat.iloc[i])
    print ("Predicted label:",  predicted_label + "\n")

hobbit picture  four years away  lord of the rings ...
Actual label: entertainment
Predicted label: entertainment

game firm holds  cast  auditions video game firm b ...
Actual label: tech
Predicted label: sport

clarke plans migrant point scheme anyone planning  ...
Actual label: politics
Predicted label: politics

radcliffe will compete in london paula radcliffe w ...
Actual label: sport
Predicted label: sport

serena becomes world number two serena williams ha ...
Actual label: sport
Predicted label: sport

ultimate game  award for doom 3 sci-fi shooter doo ...
Actual label: tech
Predicted label: tech

algeria hit by further gas riots algeria suffered  ...
Actual label: business
Predicted label: business

fast lifts rise into record books two high-speed l ...
Actual label: tech
Predicted label: tech

muslim group attacks tv drama 24 a british muslim  ...
Actual label: entertainment
Predicted label: entertainment

us tv special for tsunami relief a us television n ...
Actual label: e